In [ ]:
path = r'C:\Users\pegas\OneDrive\Desktop\pegasus-backtest-new\strategy\moving_average\moving_average_NIFTY_0930_1430_15_30_10_20_ema_tradebook.csv'

In [127]:
import pandas as pd

In [128]:
df = pd.read_csv('deepak.csv')

In [129]:
daily_returns_df = df.copy()

In [130]:
daily_returns_df.set_index('date',inplace=True)

In [131]:
daily_returns_df['cummax'] = daily_returns_df['drawdown'].cummax()


In [137]:
def calculate_streaks(series):
    streak_count = 0
    max_streak = 0
    for value in series:
        if value > 0:
            streak_count += 1
            max_streak = max(max_streak, streak_count)
        else:
            streak_count = 0
    return max_streak

In [140]:
import concurrent.futures
import pandas as pd
counter = 0
def calculate_implied_futures(idx, row):
    
    try:
        timestamp = row['timestamp']
        atm = row['atm']
        
        ce_data = ce[(ce['timestamp'] == timestamp) & (ce['strike'] == atm)]
        pe_data = pe[(pe['timestamp'] == timestamp) & (pe['strike'] == atm)]

        ce_min_expiry = min(ce_data['expiry'])
        pe_min_expiry = min(pe_data['expiry'])
        ce_max_expiry = max(ce_data['expiry'])
        pe_max_expiry = max(pe_data['expiry'])
        
        ce_close_min = ce_data[ce_data['expiry'] == ce_min_expiry]['close'].values[0]
        pe_close_min = pe_data[pe_data['expiry'] == pe_min_expiry]['close'].values[0]
        ce_close_max = ce_data[ce_data['expiry'] == ce_max_expiry]['close'].values[0]
        pe_close_max = pe_data[pe_data['expiry'] == pe_max_expiry]['close'].values[0]
        
        implied_futures_weekly = round((atm + ce_close_min - pe_close_min), 2)
        implied_futures_monthly = round((atm + ce_close_max - pe_close_max), 2)
        
        df.loc[idx, 'implied_futures_weekly'] = implied_futures_weekly
        df.loc[idx, 'implied_futures_monthly'] = implied_futures_monthly
#         df.loc[idx, 'ce_symbol_weekly'] = ce_data[ce_data['expiry'] == ce_min_expiry]['symbol'].values[0]
#         df.loc[idx, 'pe_symbol_weekly'] = pe_data[pe_data['expiry'] == pe_min_expiry]['symbol'].values[0]
#         df.loc[idx, 'ce_symbol_monthly'] = ce_data[ce_data['expiry'] == ce_max_expiry]['close'].values[0]
#         df.loc[idx, 'pe_symbol_monthly'] = pe_data[ce_data['expiry'] == pe_max_expiry]['close'].values[0]
#         df.loc[idx, 'ce_weekly_close'] = ce_close_min
#         df.loc[idx, 'pe_weekly_close']= pe_close_min
#         df.loc[idx, 'ce_monthly_close'] = ce_close_max
#         df.loc[idx, 'pe_monthly_close'] = pe_close_max
        print(f"Weekly Implied Futures: {implied_futures_weekly}, Monthly Implied Futures: {implied_futures_monthly}")
        counter+=1
        print(f'updated rows count {counter}')
    except Exception as e:
        print(f"Error processing data at timestamp: {timestamp}. Error: {e}")
df = df[:100]
with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
    futures = {executor.submit(calculate_implied_futures, idx, row): (idx, row) for idx, row in df.iterrows()}
    for future in concurrent.futures.as_completed(futures):
        idx, row = futures[future]
        try:
            future.result()
        except Exception as exc:
            print(f"Thread for row {idx} generated an exception: {exc}")
print(df)

In [142]:
max_losing_streak, max_winning_streak

(5, 7)

In [102]:
drawdown_series = daily_returns_df['drawdown'] < 0  
drawdown_series = drawdown_series.astype(int)  
drawdown_series_diff = drawdown_series.diff()  

start_indices = drawdown_series_diff[drawdown_series_diff == 1].index
end_indices = drawdown_series_diff[drawdown_series_diff == -1].index
start_indices = pd.DataFrame(start_indices, columns=['date'])
start_indices = pd.to_datetime(start_indices['date'])
end_indices = pd.DataFrame(end_indices, columns=['date'])
end_indices = end_indices.rename(columns ={'date':'end_date'})
end_indices = pd.to_datetime(end_indices['end_date'])
concatenated_df = pd.concat([start_indices, end_indices], axis=1)
concatenated_df['dd_days']=concatenated_df['end_date'] - concatenated_df['date']
max_days_in_dd = (concatenated_df.max()).dd_days
max_days_in_dd = max_days_in_dd.days
# # Check if the drawdown series starts or ends with a drawdown
# if drawdown_series.iloc[0]:
#     start_indices = start_indices.insert(0, drawdown_series.index[0])
# if drawdown_series.iloc[-1]:
#     end_indices = end_indices.insert(len(end_indices), drawdown_series.index[-1])

# # Find the lengths of consecutive negative values (drawdown)
# consecutive_lengths = end_indices - start_indices

# # Find the maximum length of consecutive negative values (drawdown)
# max_drawdown_days = consecutive_lengths.max() if not consecutive_lengths.empty else 0

# print("Max number of days in drawdown:", max_drawdown_days)


In [113]:
concatenated_df

,date,end_date,dd_days
0,2021-01-06,2021-01-08,2 days
1,2021-01-13,2021-01-19,6 days
2,2021-01-29,2021-02-01,3 days
3,2021-02-05,2021-02-08,3 days
4,2021-02-12,2021-08-03,172 days
5,2021-08-06,2021-08-13,7 days
6,2021-08-17,2021-08-31,14 days
7,2021-09-01,2021-09-02,1 days
8,2021-09-06,2021-09-15,9 days
9,2021-09-17,2021-09-21,4 days


In [115]:
concatenated_df.dropna(inplace=True)

In [123]:
daily_returns_df

,Unnamed: 0,pnl,pnl_pct,pnl_pct_cumulative,drawdown,cummax
date,,,,,,
2021-01-04,0,660.0,0.33000,0.33000,0.000000,0.0
2021-01-05,1,2635.0,1.31750,1.64750,0.000000,0.0
2021-01-06,2,-1292.5,-0.64625,1.00125,-39.226100,0.0
2021-01-07,3,-1040.0,-0.52000,0.48125,-70.789074,0.0
2021-01-08,4,2375.0,1.18750,1.66875,0.000000,0.0
...,...,...,...,...,...,...
2021-12-23,182,392.5,0.19625,45.29875,-15.920744,0.0
2021-12-24,183,-2870.0,-1.43500,43.86375,-18.584256,0.0
2021-12-27,184,3622.5,1.81125,45.67500,-15.222385,0.0


172

In [14]:
drawdown_series = daily_returns_df['drawdown'] < 0  
drawdown_series = drawdown_series.astype(int) 
drawdown_series_diff = drawdown_series.diff()  

start_indices = drawdown_series_diff[drawdown_series_diff == 1].index
end_indices = drawdown_series_diff[drawdown_series_diff == -1].index

consecutive_lengths = end_indices - start_indices

max_drawdown_days = consecutive_lengths.max() if not consecutive_lengths.empty else 0

# print("Max number of days in drawdown:", max_drawdown_days)


ValueError: operands could not be broadcast together with shapes (14,) (15,) 

In [143]:
def calculate_cagr(initial_capital, net_pnl):
    ending_value = initial_capital + net_pnl
    number_of_years = 1
    
    cagr = (ending_value / initial_capital) ** (1 / number_of_years) - 1
    return cagr


In [144]:
calculate_cagr(100,50)

0.5

In [ ]:
df.drawdown

In [ ]:
def trade_analysis(df,lot_size,brokerage):
    total_trades = len(df)
    winning_trades = len(df[df.pnl >= 0])
    losing_trades = len(df[df.pnl < 0])
    net_profit = round(sum(df['pnl'])*lot_size,2)
    winning_strike = round(winning_trades / losing_trades if losing_trades != 0 else 0,2)  
    avg_profit = round(sum(df[df.pnl >= 0].pnl) / winning_trades if winning_trades != 0 else 0,2 ) 
    avg_loss = round(sum(df[df.pnl < 0].pnl) / losing_trades if losing_trades != 0 else 0,2)  
    total_brokerage = round(total_trades * brokerage)
    return total_trades, winning_trades, losing_trades, net_profit, winning_strike, avg_profit, avg_loss, total_brokerage


In [ ]:
trade_analysis(df,50,40)

In [ ]:
df

In [ ]:
df['date'] = pd.to_datetime(df.entry_time).dt.date.copy()

In [ ]:
df

In [ ]:
def calculate_daily_pnl(df):
    data = []
    tradebook_grp = df.groupby('date')
    for date, grp in tradebook_grp:
        day_pnl = grp['pnl'].sum()
        data.append({'date': date, 'pnl': day_pnl})
    daily_pnl = pd.DataFrame(data)
    return daily_pnl

In [ ]:
daily_returns = calculate_daily_pnl(df)

In [ ]:
capital = 200000
lot_size= 50

In [ ]:
daily_returns_df.pnl = daily_returns_df.pnl * lot_size
daily_returns_df["pnl_pct"] = (daily_returns_df["pnl"] / capital) * 100
# daily_returns_df.to_csv('daily_returns.csv')
daily_returns = daily_returns_df.copy()
daily_returns['date'] = pd.to_datetime(daily_returns['date']).dt.date
daily_pnl = daily_returns.groupby('date')['pnl'].sum().reset_index()
# daily_pnl.to_csv('daily_returns.csv',index=False)

In [ ]:
daily_pnl

In [ ]:
daily_returns_df = daily_returns.copy()

In [ ]:
daily_returns

In [ ]:
daily_returns_df.reset_index(inplace=True)
daily_returns_df["pnl_pct_cumulative"] = daily_returns_df["pnl_pct"].cumsum()
capital = 200000
cumulative_pnl = np.cumsum(daily_returns_df['pnl'])
max_cumulative_pnl = np.maximum.accumulate(cumulative_pnl)
daily_returns_df["drawdown"] = ((cumulative_pnl / max_cumulative_pnl) - 1) * 100

In [ ]:
    def calculate_daily_returns(self, capital=200000):
        self.daily_returns_df = self.daily_pnl.copy()
        self.daily_returns_df.pnl = self.daily_returns_df.pnl * 15
        self.daily_returns_df["pnl_pct"] = (self.daily_returns_df["pnl"] / capital) * 100
        self.daily_returns_df.to_csv('daily_returns.csv')
        daily_returns = self.daily_returns_df.copy()
        daily_returns['date'] = daily_returns['date'].dt.date
        daily_pnl = daily_returns.groupby('date')['pnl'].sum().reset_index()
        daily_pnl.to_csv('daily_returns.csv',index=False)

In [ ]:
# from reportlab.lib import colors as reportlab_colors

# def save_trade_analysis_to_pdf(df, lot_size, brokerage, filename="trade_analysis.pdf"):
#     # Perform trade analysis
#     total_trades, winning_trades, losing_trades, net_profit, winning_strike, avg_profit, avg_loss, total_brokerage = trade_analysis(df, lot_size, brokerage)
    
#     # Create bar chart
#     labels = ['Winning Trades', 'Losing Trades']
#     values = [winning_trades, losing_trades]
#     colors = ['#4CAF50', '#FF5733']  # Green for winning, Red for losing
#     plt.figure(figsize=(8, 6))
#     plt.bar(labels, values, color=colors)
#     plt.title('Winning vs Losing Trades')
#     plt.xlabel('Trade Outcome')
#     plt.ylabel('Number of Trades')
#     plt.savefig('trade_outcome.png')  # Save the bar chart as a PNG
    
#     # Creating the PDF
#     pdf = SimpleDocTemplate(filename, pagesize=letter)
    
#     # Create data for table
#     data = [
#         ["Total Trades", total_trades],
#         ["Winning Trades", winning_trades],
#         ["Losing Trades", losing_trades],
#         ["Net Profit", f"${net_profit}"],
#         ["Winning Strike", winning_strike],
#         ["Avg. Profit", f"${avg_profit}"],
#         ["Avg. Loss", f"${avg_loss}"],
#         ["Total Brokerage", f"${total_brokerage}"]
#     ]
    
#     # Adding table to PDF
#     table = Table(data)
#     style = TableStyle([('BACKGROUND', (0, 0), (-1, 0), reportlab_colors.gray),
#                         ('TEXTCOLOR', (0, 0), (-1, 0), reportlab_colors.whitesmoke),
#                         ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
#                         ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
#                         ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
#                         ('BACKGROUND', (0, 1), (-1, -1), reportlab_colors.beige),
#                         ('GRID', (0, 0), (-1, -1), 1, reportlab_colors.black)])
#     table.setStyle(style)
#     pdf.build([table])
    

In [ ]:
def calculate_cagr(initial_capital, final_gain, number_of_years=1):
    final_value = initial_capital + final_gain
    cagr = (final_value / initial_capital) ** (1 / number_of_years) - 1
    return round(cagr*100,2)

In [ ]:
calculate_cagr(200000,90585)

In [ ]:
trade_analysis(df,50,40)

In [ ]:
save_trade_analysis_to_pdf(df,50, 40)